In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pickle


In [ ]:
book = pd.read_csv("data/Books.csv")
book.shape

/tmp/ipykernel_74712/2850078242.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv("data/Books.csv")


(278858, 3)

In [10]:
ratings = pd.read_csv("data/Ratings.csv")
ratings.shape


(1149780, 3)

In [11]:
user = pd.read_csv("data/Users.csv")
user.shape

(278858, 3)

In [13]:
book.isnull().sum()

ISBN           0
Book-Title     0
Book-Author    0
Publisher      0
dtype: int64

In [15]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [33]:
user.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [37]:
book.duplicated().sum()


np.int64(0)

In [38]:
ratings.duplicated().sum()


np.int64(0)

In [39]:
user.duplicated().sum()


np.int64(0)

In [40]:
ratings_with_name=ratings.merge(book, on="ISBN")
ratings_with_name.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose
1,276726,0155061224,5,Rites of Passage,Judith Rae
2,276727,0446520802,0,The Notebook,Nicholas Sparks
3,276729,052165615X,3,Help!: Level 1,Philip Prowse
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather


In [41]:
num_ratings_df=ratings_with_name.groupby("Book-Title").count()["Book-Rating"].reset_index()

In [46]:
num_ratings_df.head()

,Book-Title,num-ratings
0,A Light in the Storm: The Civil War Diary of ...,1
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [43]:
num_ratings_df.rename(columns={"Book-Rating":"num-ratings"}, inplace=True)
num_ratings_df

,Book-Title,num-ratings
0,A Light in the Storm: The Civil War Diary of ...,1
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241060,Ã?Â?lpiraten.,1
241061,Ã?Â?rger mit Produkt X. Roman.,1
241062,Ã?Â?sterlich leben.,1
241063,Ã?Â?stlich der Berge.,1


In [69]:
avg_rating_df=ratings_with_name.groupby('Book-Title')['Book-Rating'].agg(lambda x: x.astype(float).mean()).reset_index()
avg_rating_df.rename(columns = {'Book-Rating':'avg_rating'}, inplace=True)
avg_rating_df

,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,0.0
1,Always Have Popsicles,0.0
2,Apple Magic (The Collector's series),0.0
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.0
4,Beyond IBM: Leadership Marketing and Finance ...,0.0
...,...,...
241060,Ã?Â?lpiraten.,0.0
241061,Ã?Â?rger mit Produkt X. Roman.,6.0
241062,Ã?Â?sterlich leben.,7.0
241063,Ã?Â?stlich der Berge.,8.0


In [70]:
popular_df=num_ratings_df.merge(avg_rating_df, on="Book-Title")

In [71]:
popular_df.head()

,Book-Title,num-ratings,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,1,0.0
1,Always Have Popsicles,1,0.0
2,Apple Magic (The Collector's series),1,0.0
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.0
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.0


In [74]:
popular_df = popular_df.sort_values("avg_rating", ascending=False)

In [75]:
popular_df

,Book-Title,num-ratings,avg_rating
50027,Diana in Private: The Princess Nobody Knows,1,10.0
152522,Russian Literature: A Very Short Introduction ...,1,10.0
152573,Ryan Blake'S Revenge (Silhouette Intimate Mome...,1,10.0
152572,Rx for Worry: A Thankful Heart,1,10.0
152565,Ruthless Trust : The Ragamuffin's Path to God,1,10.0
...,...,...,...
140546,Pineapple Tart,1,0.0
140543,Pine Tree Parable Board Book,1,0.0
140542,"Pinckert's practical grammar: A lively, uninti...",1,0.0
140541,Pincher Martin: The Two Deaths of Christopher ...,1,0.0


In [76]:
popular_df=popular_df.merge(book, on="Book-Title")[["Book-Title","Book-Author","num-ratings","avg_rating"]]
popular_df

,Book-Title,Book-Author,num-ratings,avg_rating
0,Diana in Private: The Princess Nobody Knows,Lady Coln Campbell,1,10.0
1,Russian Literature: A Very Short Introduction ...,Catriona Kelly,1,10.0
2,Ryan Blake'S Revenge (Silhouette Intimate Mome...,Clara Wimberly,1,10.0
3,Rx for Worry: A Thankful Heart,James P. Gills,1,10.0
4,Ruthless Trust : The Ragamuffin's Path to God,Brennan Manning,1,10.0
...,...,...,...,...
270284,Pineapple Tart,Anne Dunlop,1,0.0
270285,Pine Tree Parable Board Book,Liz Curtis Higgs,1,0.0
270286,"Pinckert's practical grammar: A lively, uninti...",Robert C Pinckert,1,0.0
270287,Pincher Martin: The Two Deaths of Christopher ...,William Golding,1,0.0


In [78]:
ratings_with_name=ratings.merge(book,on="ISBN")

In [79]:
ratings_with_name.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose
1,276726,0155061224,5,Rites of Passage,Judith Rae
2,276727,0446520802,0,The Notebook,Nicholas Sparks
3,276729,052165615X,3,Help!: Level 1,Philip Prowse
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather


In [80]:
x=ratings_with_name.groupby('User-ID').count()["Book-Rating"]>200
users_with_high_num_of_ratings=x[x].index
users_with_high_num_of_ratings

Index([   254,   2276,   3363,   4385,   6251,   6543,   6575,   7158,   7346,
         8681,
       ...
       245963, 248718, 251613, 261105, 262998, 269566, 271284, 277427, 277639,
       278418],
      dtype='int64', name='User-ID', length=170)

In [81]:
filtered_rating=ratings_with_name[ratings_with_name["User-ID"].isin(users_with_high_num_of_ratings)]

In [82]:
filtered_rating

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author
1122,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll
1123,277427,003008685X,8,Pioneers,James Fenimore Cooper
1124,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau
1125,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens
1126,277427,0060006641,10,"On Writing Well, 25th Anniversary : The Classi...",William Zinsser
...,...,...,...,...,...
267967,271284,1574881515,0,This Kind of War the Classic Korean War,T. R. Fehrenbach
267968,271284,1574882597,0,This Kind of War: The Classic Korean War History,T. R. Fehrenbach
267969,271284,1575661780,0,The Ex,John Lutz
267970,271284,1586631896,0,"Fly Navy: Naval Aviators and Carrier Aviation,...",Phillip Kaplan


In [155]:
y=filtered_rating.groupby('Book-Title').count()['Book-Rating']>3
famous_books=y[y].index

In [156]:
famous_books

Index(['A Little Princess', 'Aftermath', 'Alice's Adventures in Wonderland',
       'All Things Bright and Beautiful', 'Archangel', 'Ashes to Ashes',
       'Beloved', 'Best Friends', 'Between Friends', 'Billy',
       ...
       'The Wind in the Willows', 'The Wonderful Wizard of Oz', 'To Die for',
       'To Love a Stranger', 'Triplets', 'Unnatural Causes', 'Warrior',
       'Wings', 'Wuthering Heights',
       'Zoids Chaotic Century (Zoids: Chaotic Century (Graphic Novels))'],
      dtype='object', name='Book-Title', length=136)

In [157]:
final_ratings=filtered_rating[filtered_rating["Book-Title"].isin(famous_books)]

In [158]:
final_ratings

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author
1145,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett
1278,277427,0425169871,0,Firebird,Janice Graham
1288,277427,0440170796,0,The Promise,Danielle Steel
1290,277427,0440176484,0,Secrets,DANIELLE STEEL
1418,277427,0679459626,0,Flesh and Blood,Jonathan Kellerman
...,...,...,...,...,...
256118,248718,031202164X,8,Ashes to Ashes,Mary Monica Pulver
258465,251613,0517618842,0,Black Beauty,Anna Sewell
258482,251613,0671440721,0,Dinosaurs,Mary Packard
258514,251613,0786003626,0,Redemption,Annee Cartier


In [159]:
final_ratings.drop_duplicates()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author
1145,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett
1278,277427,0425169871,0,Firebird,Janice Graham
1288,277427,0440170796,0,The Promise,Danielle Steel
1290,277427,0440176484,0,Secrets,DANIELLE STEEL
1418,277427,0679459626,0,Flesh and Blood,Jonathan Kellerman
...,...,...,...,...,...
256118,248718,031202164X,8,Ashes to Ashes,Mary Monica Pulver
258465,251613,0517618842,0,Black Beauty,Anna Sewell
258482,251613,0671440721,0,Dinosaurs,Mary Packard
258514,251613,0786003626,0,Redemption,Annee Cartier


In [160]:
pt=final_ratings.pivot_table(index="Book-Title",columns="User-ID",values="Book-Rating")
pt.fillna(0, inplace=True)

In [161]:
pt.head()

User-ID,254,2276,3363,6251,6575,7158,7346,8681,10819,11601,...,235842,238781,245827,245963,248718,251613,271284,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
A Little Princess,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aftermath,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alice's Adventures in Wonderland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All Things Bright and Beautiful,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Archangel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [162]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores=cosine_similarity(pt)

In [163]:
import pickle

pickle.dump(
    {
        "pt": pt,
        "similarity_scores": similarity_scores,
        "book": book
    },
    open("recommender.pkl", "wb")
)
